# Data Manipulation in Pandas

## Skills

1. Load data into Python using the pandas module.
2. Select columns using `[]` and rows using `DataFrame.loc[]`.
3. Summarize columns with basic descriptive statistics.
4. **Summarize by category using `DataFrame.groupby()`**
5. **Create new columns.**
6. **Use built-in Pandas string manipulation functions.**
7. Visualize data using Seaborn and Matplotlib.

## Vocabulary List

**indicator variable**. A column in a dataset which takes on only *True/False* values. More specifically, we will be using indicator variables in this class to store information about another column, for example:

* "Is this person taller than 6'?" is a summary of another column, a peron's height.
* "Is this movie over 2 hours long?" is a summary of another column, the length of a movie.

## Preliminaries

Run the following code to load in the *pandas* and *matplotlib.pyplot* libraries, as well as load in the Netflix dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
netflix = pd.read_csv("netflix.csv")

netflix.head(3)

## The `.groupby()` Function

Often, we won't just want overall statistics, but statistics for each category. For example, we might ask whether movies or TV shows tend to have higher IMDB user ratings. We *can* calculate this with the skills from the previous notebook, but it is rather tedious.

First, we select all movies, and calculate their average IMDB score:

In [ ]:
netflix[ netflix["type"] == "SHOW" ]["imdb_score"].mean()

Then, we select all TV shows, and calculate their average IMDB score:

In [ ]:
netflix[ netflix["type"] == "MOVIE" ]["imdb_score"].mean()

Now imagine doing this when there are 5, 10, or even more categories, for example: what is the average IMDB user rating for movies from each year. Clearly this is inconvenient!

The `DataFrame.groupby()` gives us a way to do this quickly and efficiently. `.groupby()` tells python to not consider individual entries in the dataframe, but to group them up based on one particular column. Since we want to get the average IMDB score based on whether it's a movie or TV show, we'll group by the `type` column:

In [ ]:
netflix.groupby("type")

There is no output when you use `.groupby()`. Instead, *pandas* is waiting for us to run some summary function on it, for example:

In [ ]:
netflix.groupby("type").mean()

Since we were interested in just the IMDB user score, we can just select that one column using the indexing brackets `[]`, and we're done:

In [ ]:
netflix.groupby("type").mean()["imdb_score"]

**Example.** As suggested above, let's calculate the average IMDB rating for movies, broken down by release year. So first, we'll narrow down the Netflix titles to movies only:

In [ ]:
movies = netflix[ netflix["type"] == "MOVIE" ]

Then, we'll group by year released:

> `movies.groupby("release_year")`

And then calculate the mean IMDB ratings:

In [ ]:
movies.groupby("release_year").mean()["imdb_score"]

As a preview of next time, let's actually plot those to see if theres a trend in ratings over time:

In [ ]:
# Calculate ratings and put in a variable
ratings = movies.groupby("release_year").mean()["imdb_score"]

# Make a plot
sns.scatterplot(x=ratings.index, y=ratings)

plt.ylim(5, 8.5)
plt.xlabel("Year")
plt.ylabel("Average IMDB User Rating")

plt.show()

It looks like there's quite a bit of scatter before the year 1990, but there's a slow downward trend in the average user rating over time.

This could be due to many reasons, which we cannot tell from the plot:
* Netflix only purchases older movies if they're very good
* The newest movies are all Netflix originals, which may not be as good as theatrical releases.
* Movies are getting worse over time.
* IMDB users love old movies.

## Modifying and Creating New Columns

Rarely will a dataset have all of the information in it that we want already available. Looking at the Netflix dataset:

* We have total runtime in minutes, but may want to break it down into hours and minutes.
* Instead of IMDB and TMDB scores separately, we may want the average scores.
* A column which indicates whether a movie was made this century, or if it is from the US or is "foreign."
* The production countries and genres are lists, while it would be much easier to have only a single value per column.
* When we get further into text analysis, we'll want to tokenize the descriptions and titles, rather than leaving them as strings.

Other issues we may see in other datasets may include:

* Columns with dollar amounts will often be read in as strings (e.g. "$4.25") and need to be converted to floating-point numbers (4.25).
* We may want to split full names into first and last names, or remove titles like "Dr." or "Mr." from them.

Let's start by adding a new column telling us the runtime of a movie or show in hours. The following code does this:

In [ ]:
netflix["runtime_hours"] = netflix["runtime"] // 60

As you can see, adding a new column based on another one is simple: you just pretend that it already exists, and assign it a value. You can do standard mathematical operations (`+`, `-`, `*`, `/`, `//`, `%`), combining multiple columns as necessary. In this case, we used integer division `//` to divide the runtime in minutes by 60, and dropped any remainder.

There is now a `runtime_hours` column in our DataFrame:

In [ ]:
netflix.columns

We can also add a `runtime_minutes` column, and make sure that everything is working as we expect.

In [ ]:
netflix["runtime_minutes"] = netflix["runtime"] % 60

netflix[ ["runtime", "runtime_hours", "runtime_minutes"] ].head(10)

**Example.** Let's add a column which is the average review score, rather than keeping both the IMDB and TMDB user scores.

In [ ]:
netflix["score"] = ( netflix["imdb_score"] + netflix["tmdb_score"] ) / 2

netflix[ ["imdb_score", "tmdb_score", "score"] ].head(6)

**Example.** Let's add an **indicator column** about the length of titles on Netflix which tells us which ones are "long." For our purposes, we'll consider a movie is "long" if it is over 2 hours in length.

Indicator columns can be added by writing any logical expression about another column, using `>`, `<`, `==`, `!=`, `>=`, `<=` and so on.

In [ ]:
netflix["is_long"] =   netflix["runtime_hours"] >= 2

In [ ]:
netflix[ ["runtime", "is_long"] ].head(6)

This indicator column is useful because we can now see if longer movies have different ratings from shorter movies:

In [ ]:
movies = netflix.loc[ netflix["type"] == "MOVIE" ]
movies.groupby("is_long").describe()["score"]

It looks like they're quite similar.

**Example.** Let's add an indicator column telling us whether a title is highly rated. Looking at the statistics for movies, it appears that 75% of all titles have a rating of 7.05 or lower, so we'll use that as the cut-off between regular movies and "highly-rated" ones.

In [ ]:
netflix.describe()["score"]

In [ ]:
netflix["highly_rated"] = netflix["score"] > 7.05

In [ ]:
netflix[ ["score", "highly_rated"] ].head(8)

## Built-In DataFrame Functions

The above section applies if you're using Python's built-in mathematical operators (`+`, `-`, and so on) and logical operators (`<`, `>`, &c.). But most of the functions for string manipulation we've used and will be using *will not work*. The reason is simple, but not obvious if you're not very familiar with Python: these functions are designed to work on a single string, float, or whatever, but not for a list or Series of them.

Consider the following, which works:

In [ ]:
str.lower("HELLO")

And this, which does not:

In [ ]:
str.lower(["HELLO", "HOW", "ARE", "YOU"])

Similarly, this also produces an error because, like *lists*, a Pandas *Series* object is a collection of values:

In [ ]:
str.lower(netflix["title"])

There are two solutions to this problem:
* Pandas has many built-in functions which duplicate string functions we've already seen. A complete list can be found [in the documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.lower.html)
* Use the `DataFrame.map()` function, which we'll use to create our own more specialized data manipulation later.

Let's look over a few useful built-in string functions in Pandas.

First, the solution to the above example is to use `Series.str.lower()`:

In [ ]:
netflix["title"].str.lower()

**Example.** How long are titles of shows on Netflix?

This is a straightforward application of the `Series.str.len()` function. Let's start by calculating lengths

In [ ]:
netflix["title"].str.len()

Now that we have this, we can simply use `.describe()` to calculate typical values:

In [ ]:
netflix["title"].str.len().describe()

The mean length is 18 characters. The longest title is 104 characters, while the shortest is 1 character long. What are these? To do this, we use `.loc[]`.

In [ ]:
netflix["title"].loc[ netflix["title"].str.len() == 1 ]

In [ ]:
netflix["title"].loc[ netflix["title"].str.len() == 104 ]

**Example.** How many dramas and comedies are on Netflix?

The `Series.str.contains()` function will indicate whether a string contains text, similar to `.find()`. We can use this on the *genres* columns to convert them to indicators.

In [ ]:
netflix["is_comedy"] = netflix["genres"].str.contains("comedy")
netflix["is_drama"]  = netflix["genres"].str.contains("drama")

In [ ]:
netflix[ ["genres", "is_comedy", "is_drama"] ].head(5)

And then count them. It looks like over 900 of the titles are *both* comedy and drama!

In [ ]:
netflix.value_counts(["is_comedy", "is_drama"])